# Tools, API & Microservices

Now that we have seen the power of prompts and a look how they come together in a simple agent, lets explore formally a few other concepts.

1. Function calling
2. Tool Calling
3. Introduction to Agents
4. Agents calling tools
5. Agentic Patterns
6. Agents and Microservices

_Each module is typically dependent on the prior modules having been completed successfully_


# Agent

Agents are the heart of complex AI applications. They combine inference, memory, safety, and tool usage into coherent workflows. At its core, an agent follows a sophisticated execution loop that enables multi-step reasoning, tool usage, and safety checks.
Ref: https://llama-stack.readthedocs.io/en/latest/building_applications/agent_execution_loop.html

An Agentic-AI eco system is much larger than the LLM/s which it uses. While LLMs are being used, the agentic structure helps us to automate using those sophisticated prompts that we talked about.  The power of agentic AI is not only in the model, but in the orchestration—how you structure the workflow to get durable, repeatable outcomes without hand-holding.


### Goal-Oriented Looping

- A raw LLM gives one-shot answers. An agent keeps trying, planning multiple steps, checking for errors, adapting.
- Think of it as: “Try → Check → Revise → Retry → Finish” or "Thought → Action → Observation → Repeat → Answer"
- The loop itself enforces discipline and depth.
- Without that structure, the LLM might shortcut the process.

### Memory & Scratchpad

Agents can keep track of:
- What they’ve tried
- What the intermediate results were
- What the user originally wanted
- LLM alone doesn’t track history or outcomes unless explicitly given.

### Tool Use

- Agents can call APIs, browse docs, or query databases. LLM alone hallucinates data. An agent says: “I don’t know—let me look it up.”

### Decomposition

- Agents break big tasks into smaller ones.
- LLMs can do this, but often need a prompt to do so.
- Agents automate that “thinking out loud.”




As we go intot the Agents Module we need a new Python dependancy

In [7]:
%pip install openai-agents==0.0.13


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import openai
import re
import httpx
import os
import requests
import rich
import json
from openai import OpenAI
import requests

os.environ["OPENAI_API_KEY"] = "dummy_key" 
api_key = "placeholder" 
base_url = "http://localhost:11434/v1/"
model = "llama3.2:3b-instruct-fp16" 


from rich import print
print("[green] Model setup[/green]")

 Model setup

## Agent calling tools
1. Simply demonstrates an agent using a tool.
1. Look at the brevity of the code compared to doing a function calling all on our own.
1. Play with the question that can be asked to agent to see how it can handle questions that may or may not require the tool

In [19]:
from agents import Agent, ModelSettings, function_tool, Runner, AsyncOpenAI, OpenAIChatCompletionsModel

# Configure the model
model = OpenAIChatCompletionsModel( 
     model=model,
     openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key = api_key)
 )

#model = OpenAIChatCompletionsModel( 
#    model=llm,
#    #openai_client=AsyncOpenAI(base_url=base_url, api_key=openai.api_key)
#    openai_client=AsyncOpenAI(api_key=openai.api_key)
#)

In [20]:

@function_tool
def get_weather(latitude:str, longitude:str) ->str:
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

agent = Agent(
    name="Blaster",
    instructions="Answer the question asked very precisely. Please think before answering",
    model= model,
    tools=[get_weather],
)

result = await Runner.run(agent, "which is warmer now: Paris or Manila?")
print(result.final_output)

Based on the current weather, Manila is warmer now than Paris. The temperature in Manila is around 28.1°C, while 
the temperature in Paris is around 14.1°C.

In [21]:
from rich.pretty import pprint
pprint(result)

RunResult(
│   input='which is warmer now: Paris or Manila?',
│   new_items=[
│   │   ToolCallItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer the question asked very precisely. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7f5389501610>,
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=None,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None,
│   │   │   │   │   reasoning=None,
│   │   │   │   │   metadata=None,
│   │   │   │   │   store=None,
│   │   │   │   │   include_usage=None,
│   │   │   │   │   extra_query=None,
│   │   │   │   │   extra_body=None,
│   │   │   │   │   extra_headers=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   │   │   │   │   'longitude': {'title': 'Longitude', 'type': 'string'}
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   'required': ['latitude', 'longitude'],
│   │   │   │   │   │   │   'title': 'get_weather_args',
│   │   │   │   │   │   │   'type': 'object',
│   │   │   │   │   │   │   'additionalProperties': False
│   │   │   │   │   │   },
│   │   │   │   │   │   on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f53895ea020>,
│   │   │   │   │   │   strict_json_schema=True
│   │   │   │   │   )
│   │   │   │   ],
│   │   │   │   mcp_servers=[],
│   │   │   │   mcp_config={},
│   │   │   │   input_guardrails=[],
│   │   │   │   output_guardrails=[],
│   │   │   │   output_type=None,
│   │   │   │   hooks=None,
│   │   │   │   tool_use_behavior='run_llm_again',
│   │   │   │   reset_tool_choice=True
│   │   │   ),
│   │   │   raw_item=ResponseFunctionToolCall(
│   │   │   │   arguments='{"latitude":"48.8566","longitude":"-2.3508"}',
│   │   │   │   call_id='call_1ztsxmzg',
│   │   │   │   name='get_weather',
│   │   │   │   type='function_call',
│   │   │   │   id='__fake_id__',
│   │   │   │   status=None
│   │   │   ),
│   │   │   type='tool_call_item'
│   │   ),
│   │   ToolCallItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer the question asked very precisely. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7f5389501610>,
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=None,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None,
│   │   │   │   │   reasoning=None,
│   │   │   │   │   metadata=None,
│   │   │   │   │   store=None,
│   │   │   │   │   include_usage=None,
│   │   │   │   │   extra_query=None,
│   │   │   │   │   extra_body=None,
│   │   │   │   │   extra_headers=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   │

## Optional - explore the Agent's "working"

Printing the result with `pprint` will allow you to explore the Agent's working

# Tools
- We are definining tools with an adornment here.
- MCP Servers are formalizing this much more and everyone is adopting this.
- This is covered in another lab.

# Agentic Patterns


We explore below 3 agentic paradigms which are widely used


### Agents Collaborating
1. Simply demonstrates an agent reviewing the work of another agent - much like a human being.
1. This is one of the primary reasons while the agents can help increase accuracty of the answer and smaller models using agents can outperform larger models without agents.
1. This pattern can be used in lots of scenarios.

   ![Collaboration Pattern](resources/images/agent_collaborate.png)
   <img src="resources/images/agent_collaborate.png" alt="description" width="300" height="200">

### Agents routing
1. Simply demonstrates an agent routing work to other agents.
1. This is a very common agentic pattern.
1. In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.
   ![pattern-1](resources/images/agent_supervisor_pattern.png)  
   ![pattern-2](resources/images/agent_hierarchical.png) 

### Agents Deterministic Workflow
1. Simply demonstrates agents calling other agents to complete a well defined workflow.
1. This is a very common agentic pattern.
1. This pattern or its variants can be put to lot of practical use and it could be combined with the collaborative pattern
   ![Workflow Pattern](resources/images/agent_plan_execute.png)

There are other agentic patterns as well which we do not cover here. But these basic concepts should help adopting other patterns much simpler.

_The graphics have been used from [langraph tutorial](https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials)_

## Agents Collaborating
1. Simply demonstrates an agent reviewing the work of another agent - much like a human being.
1. This is one of the primary reasons while the agents can help increase accuracty of the answer and smaller models using agents can outperform larger models without agents.
1. This pattern can be used in lots of scenarios.

## Agents routing
1. Simply demonstrates an agent routing work to other agents.
1. This is a very common agentic pattern.
1. Ask the question in German and see what happens! In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.

## Agents Deterministic Workflow
1. Simply demonstrates agents calling other agents to complete a well defined workflow.
1. This is a very common agentic pattern.
1. This pattern or its variants can be put to lot of practical use and it could be combined with the collaborative pattern.

In [22]:
@function_tool
def get_dependency(service:str) ->list:
    dep_service=["foo","bar","baz"]
    return dep_service

did_agent = Agent(
    name="DependencyIdentifier Agent",
    instructions="An indicident will be passed on. From that, firstly identify the affected service name only. Next, identify what are the service dependencies for that service. Just return all service names. And nothing else",
    model= model,
    tools=[get_dependency],
)

#dep_result = await Runner.run(did_agent, "Incident: Shopping cart response time has increased to 10 sec")
#print(dep_result.final_output)

In [23]:
@function_tool
def get_changelog(service:list) ->list:
    change_log=["foo changed","bar changed"]
    return change_log

change_agent = Agent(
    name="ChangeLog Agent",
    instructions="An array of service names will be passed on. Identify what has changed with these services and return them. Do not return duplicate changes",
    model= model,
    tools=[get_changelog],
)

#change_result = await Runner.run(change_agent, "Shopping cart, foo, bar, baz")
#print(change_result.final_output)

In [24]:
@function_tool
def get_errorlog(service:list) ->list:
    error_log=["foo is responding slowly"]
    return error_log

error_agent = Agent(
    name="Error Log Agent",
    instructions="An array of service names will be passed on. Identify which of these services may have error messages in the logs",
    model= model,
    tools=[get_errorlog],
)

#error_result = await Runner.run(error_agent, "Shopping cart, foo, bar, baz")
#print(error_result.final_output)

In [25]:
debugger_agent = Agent(
    name="Debugger Agent",
    instructions="You will be given: 1. Incident details. 2. Services that could have been root cause of the problem. 3. Services that were changed in the time interval. 4. Services that had errors in the logs. Based on the above, loigically think through and conclude the most likely reason for this problem. Please lay down your thought process clearly that led you to the conclusion. ",
    model= model
)


In [26]:
import asyncio
async def orchestrate(input):
    # Call the intermediate agents to gather the facts
    # These all use tools heavily
    dep_result = await Runner.run(did_agent,input)
    change_result = await Runner.run(change_agent, dep_result.final_output)
    error_result = await Runner.run(error_agent, dep_result.final_output)

    services = dep_result.final_output               # e.g. ["foo","bar","baz"]
    changes  = change_result.final_output             # e.g. ["foo changed","bar changed"]
    errors   = error_result.final_output              # e.g. ["foo is responding slowly"]

    # Build a single prompt string:
    message = (
        f"Incident details: Incident: Shopping cart response time has increased to 10 sec\n\n"
        f"Affected services: {', '.join(services)}\n"
        f"Changes detected: {', '.join(changes)}\n"
        f"Error logs: {', '.join(errors)}\n\n"
        "Based on the above, logically think through and conclude the most likely reason for this problem. "
        "Please lay down your thought process clearly that led you to the conclusion."
    )

    # Debugger agent - the one who does the analysis with all relevant data
    debugger_agent = Agent(
        name="Debugger Agent",
        instructions="You will be given an incident, affected services, changes and errors. "
                     "Provide a clear, step-by-step reasoning and then your final diagnosis.",
        model=model
    )

    # Invoke it:
    debugger_result = await Runner.run(debugger_agent, message)
    return debugger_result.final_output


In [27]:
input = "Incident: Shopping cart response time has increased to 10 sec"
diagnosis = await orchestrate(input)
print("=== Debugger Thought Process & Conclusion ===")
print(diagnosis)

=== Debugger Thought Process & Conclusion ===

To diagnose the cause of the increased shopping cart response time, I will follow a logical step-by-step approach.

1. **Incident Description**: The incident reports an increase in the shopping cart response time, which is 
typically measured by the time it takes for the web browser or page to load when clicking on a product and 
navigating to another website or service that depends on Tcommerce.com's services (indicated by *).

2. **Data Collection**:
    - Changes detected: The total calls and increased usage of certain resources indicate an increased workload 
across multiple processes, which is often linked to resource-intensive activities.
    - Error logs: Specific errors are mentioned, but they do not directly point to a cause for the increased 
response time; however, their existence suggests that something within the system or processes affected by 
Tcommerce.com's services may be impacted.
    
3. **Likely Culprits for Increased Response Time**:
    - **High Traffic**: The mention of "formart" (presumably a typo) in relation to high activity and traffic, 
suggesting an event like holiday shopping season peak might be responsible for the increased response time.
    - **Resource-Intensive Processes**: If changes include enhancements or increases in calculations or database 
queries within Tcommerce, it could increase resource utilization, leading to slower responses.

4. **Considering All Factors**:
    - Given that both high traffic (likely a consequence of peak shopping seasons) and increased usage of system 
resources are factors, these elements together suggest the most plausible explanation for the delayed response: 
        The system (specifically related to Tcommerce's functionality and user behavior) has seen a significant 
surge in users navigating through its extensive catalog. This scenario typically occurs during peak shopping 
periods or major updates. These scenarios often overlap; hence, it is probable that both high demand of resources 
and an excessive number of server requests have happened concurrently.

5. **Conclusion**:
    The most likely cause for the increase in shopping cart response time based on the given information appears to
be a combination of factors: 
    - High traffic, which can include user navigations within an expansive online store.
    - Resource-intensive processes possibly due to increased calculations, database queries, or server load, 
triggered by enhanced features being utilized during peak usage periods.

This analysis points towards system resource overload and capacity strain due to high demand across multiple 
fronts: the increased need for processing power and connectivity, exacerbated by a likely scenario of holiday 
shopping traffic and potential server enhancements or updates.

# Microservices

There are several meaningful similarities between LLM-based AI agents and microservices:

## Similarities
#### Specialized functionality: 
Both are designed to handle specific tasks or domains. Microservices focus on particular business capabilities, while AI agents can be specialized for specific types of interactions or knowledge domains.
### Independent operation: 
Both can operate autonomously within their defined scope. Once configured, they can process requests without requiring constant supervision.
### Communication patterns: 
Both typically communicate via messages/APIs. Microservices use REST/gRPC/messaging protocols, while AI agents receive prompts and return responses through APIs.
### Composability: 
Both can be combined to build larger systems. Microservices can be orchestrated to create complex applications; similarly, multiple AI agents can work together in a workflow.
### Statelessness vs. statefulness: 
Basic implementations of both can be stateless, but more sophisticated versions maintain state. The Agent class you showed maintains conversation history, similar to how some microservices maintain session state.
### Scaling considerations: 
Both face similar operational challenges around scaling, monitoring, and versioning.

## Key differences:

### Implementation: 
Microservices are traditional code with deterministic logic, while LLM agents use probabilistic models. MCP Servers which expose tools to be used by Agents could be totally traditional code with deterministic logic.
### Predictability: 
Microservices have more predictable outputs for given inputs, while LLM responses can vary.


# AFTERWORD
Agents are an extremely powerful construct in the field of Generative AI:
1. You can achieve complex tasks designing appropriate agents and tools and driving interaction between the different agents.
1. There are known ways by which we can improve accuracy of the output. Much like human beings help check one another's work, agents can do the same.
1. External data retrieval and queries are carried out through the tools.
1. If agent processing needs to be vetted, make sure humans are used (human-in-the-loop) to are used to vet the agent output before it moves to the next step. Really, this is no different to how we operate in our real life with human beings - we have review and approval processes etc.